In [1]:
%matplotlib inline

# char2indices, indices2char

In [70]:
text = open('nietzsche.txt').read()
chars = sorted(list(set(text)))
chars.insert(0, "\0")  # for padding
vocab_size = len(chars)

In [9]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [10]:
idx = [char_indices[c] for c in text]

In [11]:
cs=3
c1_dat = [idx[i] for i in range(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]

In [13]:
import numpy as np

x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [14]:
y = np.stack(c4_dat[:-2])

# simple model

In [33]:
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.merge import add
from keras.models import Model
from keras.optimizers import Adam

In [21]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)
n_fac = 42
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

In [45]:
dense_in = Dense(256, activation='relu')
dense_hidden = Dense(256, activation='relu')

c1_hidden = dense_in(c1)

c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = add([c2_dense, hidden_2])

c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = add([c3_dense, hidden_3])

dense_out = Dense(vocab_size, activation='softmax')
c4_out = dense_out(c3_hidden)

In [46]:
model = Model([c1_in, c2_in, c3_in], c4_out)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c3 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c2 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c1 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1, 42)         3570        c3[0][0]                         
___________________________________________________________________________________________

In [50]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.fit([x1, x2, x3], y, batch_size=64, epochs=4)

Epoch 1/4
200295/200295 [==============================] - 18s - loss: 2.3414    
Epoch 2/4
200295/200295 [==============================] - 18s - loss: 2.1524    
Epoch 3/4
200295/200295 [==============================] - 18s - loss: 2.1040    
Epoch 4/4
200295/200295 [==============================] - 18s - loss: 2.0810    


In [51]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array([i]) for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]
get_next(' an')

'd'

In [52]:
get_next(' th')

'e'

In [53]:
get_next('phi')

'l'

# RNN

In [54]:
cs=8
c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cs)] for n in range(cs)]
c_out_dat = [idx[i+cs] for i in range(0, len(idx)-1-cs, cs)]
xs = [np.stack(c[:-2]) for c in c_in_dat]
y = np.stack(c_out_dat[:-2])

In [55]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [56]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [57]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)
n_fac = 42
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [59]:
dense_in = Dense(256, activation='relu')
dense_hidden = Dense(256, activation='relu', kernel_initializer='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [62]:
hidden = dense_in(c_ins[0][1])   # c_ins[0][1] = flattened embedding of the first character
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = add([c_dense, hidden])
c_out = dense_out(hidden)

In [63]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [64]:
model.fit(xs, y, batch_size=64, epochs=12)

Epoch 1/12
75109/75109 [==============================] - 11s - loss: 2.5333    
Epoch 2/12
75109/75109 [==============================] - 11s - loss: 2.2549    
Epoch 3/12
75109/75109 [==============================] - 11s - loss: 2.1539    
Epoch 4/12
75109/75109 [==============================] - 11s - loss: 2.0848    
Epoch 5/12
75109/75109 [==============================] - 11s - loss: 2.0285    
Epoch 6/12
75109/75109 [==============================] - 11s - loss: 1.9816    
Epoch 7/12
75109/75109 [==============================] - 11s - loss: 1.9401    
Epoch 8/12
75109/75109 [==============================] - 11s - loss: 1.9038    
Epoch 9/12
75109/75109 [==============================] - 11s - loss: 1.8725    
Epoch 10/12
75109/75109 [==============================] - 11s - loss: 1.8430    
Epoch 11/12
75109/75109 [==============================] - 11s - loss: 1.8157    
Epoch 12/12
75109/75109 [==============================] - 11s - loss: 1.7926    


In [65]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [66]:
get_next('for thos')

'e'

In [67]:
get_next('part of ')

't'

In [68]:
get_next('queens a')

'n'

# Keras RNN

In [71]:
n_fac, cs, vocab_size

(42, 8, 85)

In [74]:
from keras.models import Sequential
from keras.layers.recurrent import SimpleRNN

In [78]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(256, activation='relu', recurrent_initializer='identity'),
        Dense(vocab_size, activation='softmax')
    ])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 8, 42)             3570      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_21 (Dense)             (None, 85)                21845     
Total params: 101,959
Trainable params: 101,959
Non-trainable params: 0
_________________________________________________________________


In [80]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.fit(np.concatenate(xs,axis=1), y, batch_size=64, epochs=8)

Epoch 1/8
75109/75109 [==============================] - 7s - loss: 2.8040     
Epoch 2/8
75109/75109 [==============================] - 7s - loss: 2.2879     
Epoch 3/8
75109/75109 [==============================] - 7s - loss: 2.0780     
Epoch 4/8
75109/75109 [==============================] - 7s - loss: 1.9352     
Epoch 5/8
75109/75109 [==============================] - 7s - loss: 1.8298     
Epoch 6/8
75109/75109 [==============================] - 7s - loss: 1.7470     
Epoch 7/8
75109/75109 [==============================] - 7s - loss: 1.6833     
Epoch 8/8
75109/75109 [==============================] - 7s - loss: 1.6292     


In [86]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [87]:
get_next_keras('this is ')

't'

In [88]:
get_next_keras('part of ')

't'

In [89]:
get_next_keras('queens a')

'n'

# returning sequence

In [93]:
c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cs)] for n in range(cs)]
c_out_dat = [[idx[i+n] for i in range(1, len(idx)-cs, cs)] for n in range(cs)]

In [95]:
ys = [np.stack(c[:-2]) for c in c_out_dat]
[ys[n][:cs] for n in range(cs)]

[array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67]),
 array([ 1, 33,  2, 72, 67, 73,  2, 68])]

In [94]:
xs = [np.stack(c[:-2]) for c in c_in_dat]
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [97]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', kernel_initializer='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

In [98]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = add([c_dense, hidden])
    # every layer now has an output
    outs.append(dense_out(hidden))

model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c0_in (InputLayer)               (None, 1)             0                                            
____________________________________________________________________________________________________
c0_emb (Embedding)               (None, 1, 42)         3570        c0_in[0][0]                      
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 42)            0           c0_emb[0][0]                     
____________________________________________________________________________________________________
zeros (InputLayer)               (None, 42)            0                                            
___________________________________________________________________________________________

In [100]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]), 1))
zeros.shape

(75109, 42)

In [103]:
model.fit([zeros]+xs, ys, batch_size=64, epochs=12)

Epoch 1/12
75109/75109 [==============================] - 25s - loss: 20.1408 - output_loss_1: 2.7147 - output_loss_2: 2.5716 - output_loss_3: 2.5115 - output_loss_4: 2.4835 - output_loss_5: 2.4725 - output_loss_6: 2.4655 - output_loss_7: 2.4673 - output_loss_8: 2.4541    
Epoch 2/12
75109/75109 [==============================] - 24s - loss: 17.8837 - output_loss_1: 2.5155 - output_loss_2: 2.3568 - output_loss_3: 2.2308 - output_loss_4: 2.1792 - output_loss_5: 2.1552 - output_loss_6: 2.1516 - output_loss_7: 2.1605 - output_loss_8: 2.1341    
Epoch 3/12
75109/75109 [==============================] - 24s - loss: 17.2578 - output_loss_1: 2.4985 - output_loss_2: 2.3338 - output_loss_3: 2.1687 - output_loss_4: 2.0872 - output_loss_5: 2.0506 - output_loss_6: 2.0423 - output_loss_7: 2.0488 - output_loss_8: 2.0279    
Epoch 4/12
75109/75109 [==============================] - 24s - loss: 16.8847 - output_loss_1: 2.4914 - output_loss_2: 2.3243 - output_loss_3: 2.1361 - output_loss_4: 2.0360 - ou

In [104]:
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [105]:
get_nexts(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['a', 'h', 'e', 't', ' ', 's', 'n', ' ']

In [106]:
get_nexts(' part of')

[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


['a', 'o', 'r', 't', 'i', 'o', 'f', ' ']

# sequence model with keras

In [109]:
from keras.layers.wrappers import TimeDistributed

In [110]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', recurrent_initializer='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 8, 42)             3570      
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 8, 256)            76544     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 8, 85)             21845     
Total params: 101,959
Trainable params: 101,959
Non-trainable params: 0
_________________________________________________________________


In [111]:
x_rnn=np.stack(np.squeeze(xs), axis=1)
y_rnn=np.atleast_3d(np.stack(ys, axis=1))

In [116]:
x_rnn.shape, y_rnn.shape

((75109, 8), (75109, 8, 1))

In [117]:
model.fit(x_rnn, y_rnn, batch_size=64, epochs=8)

Epoch 1/8
75109/75109 [==============================] - 7s - loss: 2.4342     
Epoch 2/8
75109/75109 [==============================] - 7s - loss: 1.9967     
Epoch 3/8
75109/75109 [==============================] - 7s - loss: 1.8817     
Epoch 4/8
75109/75109 [==============================] - 7s - loss: 1.8217     
Epoch 5/8
75109/75109 [==============================] - 7s - loss: 1.7840     
Epoch 6/8
75109/75109 [==============================] - 7s - loss: 1.7571     
Epoch 7/8
75109/75109 [==============================] - 7s - loss: 1.7378     
Epoch 8/8
75109/75109 [==============================] - 7s - loss: 1.7213     


In [118]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]
get_nexts_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 's', 's', ' ']

# stateful model

In [121]:
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM

In [122]:
bs = 64
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (64, 8, 42)               3570      
_________________________________________________________________
batch_normalization_1 (Batch (64, 8, 42)               168       
_________________________________________________________________
lstm_1 (LSTM)                (64, 8, 256)              306176    
_________________________________________________________________
time_distributed_2 (TimeDist (64, 8, 85)               21845     
Total params: 331,759
Trainable params: 331,675
Non-trainable params: 84
_________________________________________________________________


In [123]:
mx = len(x_rnn)//bs*bs

In [126]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, epochs=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 27s - loss: 1.7564    
Epoch 2/4
75072/75072 [==============================] - 26s - loss: 1.7421    
Epoch 3/4
75072/75072 [==============================] - 26s - loss: 1.7291    - ETA: 0s - l
Epoch 4/4
75072/75072 [==============================] - 26s - loss: 1.7171    


# one-hot sequence

In [161]:
model=Sequential([
        SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', recurrent_initializer='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [162]:
from keras.utils.np_utils import to_categorical
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

oh_x_rnn.shape, oh_y_rnn.shape

((75109, 8, 85), (75109, 8, 85))

In [163]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, epochs=8)

Epoch 1/8
75109/75109 [==============================] - 7s - loss: 2.4442     
Epoch 2/8
75109/75109 [==============================] - 7s - loss: 2.0342     
Epoch 3/8
75109/75109 [==============================] - 7s - loss: 1.9211     
Epoch 4/8
75109/75109 [==============================] - 7s - loss: 1.8568     
Epoch 5/8
75109/75109 [==============================] - 7s - loss: 1.8141     
Epoch 6/8
75109/75109 [==============================] - 7s - loss: 1.7830     
Epoch 7/8
75109/75109 [==============================] - 7s - loss: 1.7597     
Epoch 8/8
75109/75109 [==============================] - 7s - loss: 1.7414     


In [164]:
def get_nexts_oh(inp):
    idxs = np.array([char_indices[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [165]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'c', 's', ' ']

# pure python rnn

In [127]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def sigmoid_d(x): 
    output = sigmoid(x)
    return output*(1-output)
def relu(x):
    return np.maximum(0., x)
def relu_d(x):
    return (x > 0.)*1.

relu(np.array([3.,-3.])), relu_d(np.array([3.,-3.]))

(array([ 3.,  0.]), array([ 1.,  0.]))

In [128]:
def dist(a,b):
    return pow(a-b,2)
def dist_d(a,b):
    return 2*(a-b)

In [134]:
import pdb

In [136]:
%%debug
for i in range(5):
    #pdb.set_trace()
    print(i)

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
None
> <string>(2)<module>()

ipdb> c
0
1
2
3
4


In [137]:
eps = 1e-7
def x_entropy(pred, actual): 
    return -np.sum(actual * np.log(np.clip(pred, eps, 1-eps)))
def x_entropy_d(pred, actual):
    return -actual/pred
def softmax(x):
    return np.exp(x)/np.exp(x).sum()
def softmax_d(x):
    sm = softmax(x)
    res = np.expand_dims(-sm,-1)*sm
    res[np.diag_indices_from(res)] = sm*(1-sm)
    return res

In [139]:
test_preds = np.array([0.2,0.7,0.1])
test_actuals = np.array([0.,1.,0.])
x_entropy(test_preds, test_actuals), x_entropy_d(test_preds, test_actuals)

(0.35667494393873245, array([-0.        , -1.42857143, -0.        ]))

In [141]:
softmax(test_preds), softmax_d(test_preds)

(array([ 0.28140804,  0.46396343,  0.25462853]),
 array([[ 0.20221756, -0.13056304, -0.07165452],
        [-0.13056304,  0.24870137, -0.11813832],
        [-0.07165452, -0.11813832,  0.18979284]]))

In [143]:
act=relu
act_d = relu_d
loss=x_entropy
loss_d=x_entropy_d

def scan(fn, start, seq):
    res = []
    prev = start
    for s in seq:
        app = fn(prev, s)
        res.append(app)
        prev = app
    return res

scan(lambda prev,curr: prev+curr, 0, range(5))

[0, 1, 3, 6, 10]

In [148]:
from keras.utils.np_utils import to_categorical
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

oh_x_rnn.shape, oh_y_rnn.shape

((75109, 8, 85), (75109, 8, 85))

In [150]:
inp = oh_x_rnn
outp = oh_y_rnn
n_input = vocab_size
n_output = vocab_size

In [151]:
def one_char(prev, item):
    # Previous state
    tot_loss, pre_hidden, pre_pred, hidden, ypred = prev
    # Current inputs and output
    x, y = item
    pre_hidden = np.dot(x,w_x) + np.dot(hidden,w_h)
    hidden = act(pre_hidden)
    pre_pred = np.dot(hidden,w_y)
    ypred = softmax(pre_pred)
    return (
        # Keep track of loss so we can report it
        tot_loss+loss(ypred, y),
        # Used in backprop
        pre_hidden, pre_pred, 
        # Used in next iteration
        hidden, 
        # To provide predictions
        ypred)

In [152]:
def get_chars(n):
    return zip(inp[n], outp[n])
def one_fwd(n):
    return scan(one_char, (0,0,0,np.zeros(n_hidden),0), get_chars(n))

In [158]:
# "Columnify" a vector
def col(x): 
    return x[:,np.newaxis]

def one_bkwd(args, n):
    global w_x,w_y,w_h

    i=inp[n]  # 8x86
    o=outp[n] # 8x86
    d_pre_hidden = np.zeros(n_hidden) # 256
    for p in reversed(range(len(i))):
        totloss, pre_hidden, pre_pred, hidden, ypred = args[p]
        x=i[p] # 86
        y=o[p] # 86
        d_pre_pred = softmax_d(pre_pred).dot(loss_d(ypred,y))  # 86
        d_pre_hidden = (np.dot(d_pre_hidden, w_h.T) 
                        + np.dot(d_pre_pred,w_y.T)) * act_d(pre_hidden) # 256

        # d(loss)/d(w_y) = d(loss)/d(pre_pred) * d(pre_pred)/d(w_y)
        w_y -= col(hidden) * d_pre_pred * alpha
        # d(loss)/d(w_h) = d(loss)/d(pre_hidden[p-1]) * d(pre_hidden[p-1])/d(w_h)
        if (p>0): w_h -= args[p-1][3].dot(d_pre_hidden) * alpha
        w_x -= col(x)*d_pre_hidden * alpha
    return d_pre_hidden

In [156]:
import math

scale=math.sqrt(2./n_input)
w_x = np.random.normal(scale=scale, size=(n_input,n_hidden))
w_y = np.random.normal(scale=scale, size=(n_hidden, n_output))
w_h = np.eye(n_hidden, dtype=np.float32)

In [159]:
overallError=0
alpha=0.0001
for n in range(10000):
    res = one_fwd(n)
    overallError+=res[-1][0]
    deriv = one_bkwd(res, n)
    if(n % 1000 == 999):
        print ("Error:{:.4f}; Gradient:{:.5f}".format(
                overallError/1000, np.linalg.norm(deriv)))
        overallError=0

Error:35.8079; Gradient:1.73636
Error:35.6095; Gradient:1.88083
Error:35.4611; Gradient:1.84063
Error:35.3487; Gradient:1.75219
Error:35.1858; Gradient:1.77124
Error:35.0096; Gradient:2.13609
Error:33.2007; Gradient:4.50439
Error:31.5541; Gradient:4.17145
Error:30.1903; Gradient:4.69561
Error:29.6151; Gradient:4.88102


# keras gru

In [169]:
from keras.layers.recurrent import GRU

model=Sequential([
        GRU(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [170]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, epochs=8)

Epoch 1/8
75109/75109 [==============================] - 16s - loss: 2.4032    
Epoch 2/8
75109/75109 [==============================] - 16s - loss: 1.9872    
Epoch 3/8
75109/75109 [==============================] - 16s - loss: 1.8770    
Epoch 4/8
75109/75109 [==============================] - 16s - loss: 1.8143    
Epoch 5/8
75109/75109 [==============================] - 16s - loss: 1.7711    
Epoch 6/8
75109/75109 [==============================] - 16s - loss: 1.7398    
Epoch 7/8
75109/75109 [==============================] - 16s - loss: 1.7158    
Epoch 8/8
75109/75109 [==============================] - 16s - loss: 1.6955    


In [171]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'i', 'n', ' ']